# Test Bench

Test bench is a class we created that takes in some perediction model and creates multiple instances of that model. Each instance is trained and tested on different data, it is used to easily access the quality of each model on multiple datasets.

## How Is It Used?

First define a class that has these functions:
1. `learn_from_time_series`
2. `predict`

In [1]:
class DumbPredictor:
    def __init__(self):
        pass

    def learn_from_time_series(self, ts_as_df):
        pass

    def predict(self, ts_as_df_start, how_much_to_predict):
        return ts_as_df_start[: how_much_to_predict]

After the model has been defined (including how you choose to you use each sample for learning, what optimization algorithms and so on) you can send the model to the test_bench to be used for prediction on multiple models.